### 1. 색상 공간 변환

``` python
dst = cv2.cvtColor(
    src,
    code,
    dstCn = None
)
```

In [5]:
import cv2

src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\crow.jpg")
dst = cv2.cvtColor(src, cv2.COLOR_BGR2HLS)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 2. HSV 색상 공간

- 색상(Hue)  
일반적으로 0~360 사이의 값으로 표현  
but, 8비트 이미지로눈 0~255의 값만 할당 가능  
=> 255의 절반인 0~179 범위로 사용.

    ※ **색상의 최솟값과 최댓값에서 빨간색 범위가 겹침. (원형 형태)**

- 채도(Saturation): 0~255 범위
- 명도(Value): 0~255 범위

  
``` python
# 채널 분리 함수
mv = cv2.split(
    src
)

# 채널 병합 함수
dst = cv2.merge(
    mv
)

# 리스트 반환 형식 변경
c0, c1, c2 = cv2.split(src)

# 리스트 형식 입력
dst = cv2.merge([c0, c1, c2])

# 배열 요소의 범위 설정
dst = cv2.inRange(
    src,
    lowerb, # 낮은 범위
    upperb  # 높은 범위
)
```




In [11]:
# Hue 공간 색상 검출

src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\tomato.jpg")
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)


h, s, v = cv2.split(hsv)
h_red = cv2.inRange(h, 0, 5)

dst = cv2.bitwise_and(hsv, hsv, mask=h_red)
dst = cv2.cvtColor(dst, cv2.COLOR_HSV2BGR)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

``` python
# 배열 병합 함수
dst = cv2.addWeighted(
    src1,
    alpha,
    src2,
    beta,
    gamma,
    dtype = None
)
```
배열 병합 함수 수식
dst = src1 x alpha + src2 x beta + gamma

In [19]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\tomato.jpg")
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

h, s, v = cv2.split(hsv)

orange = cv2.inRange(hsv, (8, 100, 100), (20, 255, 255))
blue = cv2.inRange(hsv, (110, 100, 100), (130, 255, 255))
mix_color = cv2.addWeighted(orange, 1.0, blue, 1.0, 0.0)

dst = cv2.bitwise_and(hsv, hsv, mask = mix_color)
dst = cv2.cvtColor(dst, cv2.COLOR_HSV2BGR)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

3. 이진화

임계값보다 낮은 픽셀값은 0이나 원본 픽셀값으로 변경, 임계값 보다 높은 픽셀값은 최댓값으로 변경
``` python
retval, dst = cv2.threshold(
    src,
    thresh, # 임계값
    maxval, # 최댓값
    type
)
```

일반적으로 **단일 채널 이미지**에서 활용  
**다중 채널 이미지**에서 이진화를 적용할 경우 각 채널을 분히해서 이진화 함수를 적용한 후 이미지를 다시 병합해서 반환

In [20]:
src = cv2.imread("C:\\Source\\openCV\\basic-openCV\\images\\swan.jpg")
_, binary = cv2.threshold(src, 127, 255, cv2.THRESH_BINARY)

cv2.imshow("binary", binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

알고리즘은 디지털 영상 처리에서 자세히 공부하기...
- 오츠 알고리즘
    - 이미지의 밝기 분포(히스토그램)을 통해 최적의 임곗값을 찾아 이진화하는 알고리즘
    - 가능한 모든 임계값을 고려해서 이미지 내의 픽셀들을 두 개의 클래스로 분류했을 때 클래스 간의 분산을 최소화하거나 차이를 최대화하는 임곗값
    - 가능한 모든 임계값을 고려하므로 느림.
    - 수식: **σ² = α x σ₁² + β x σ₂²** (이건 클래스 내의 분산 아닌가)
    - Bimoadl Image에서 사용하기 적합 (Bimoadl Image" 히스토그램상에서 두개의 피크를 치는 형태의 이미지)

        https://aliencoder.tistory.com/53
    


정확히는 모르겠음.
- 삼각형 알고리즘
    - 이미지의 밝기 분포(히스토그램)을 통해 최적의 임곗값을 찾아 이진화하는 알고리즘.
    - 히스토그램에서 최대 거리를 구성할 수 있는 임계값을 찾아 이진화를 적용.
    - 삼각형의 빗변 사이의 거리가 최대일때 수직인 선이 히스토그램의 최대 거리
    - 즉, 히스토그램에 그려진 선 사이의 거리가 최대인 지역값이 임계값
